# Step 1: Import Libraries and Load the Dataset

In [ ]:
# Imports

#pathlib: modern, clean way to handle file and folder paths
from pathlib import Path

#shutil: used to copy files from raw folders to the train and test folders
import shutil

import random

# nump useful for numerical operations
import numpy as np

import pandas as pd

#matplotlib.pyplot: uswed for visualizations
import matplotlib.pyplot as plt

#tensorflow and keras for deep learning
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Step 2: Data setup

In [1]:
# Here we first must define the original datasets paths
# then educe them to 5,000 images for train and 2,000 for test which will be appropriate for the scale of the workshop

raw_train_dir = Path("data/raw/train/train")
raw_test_dir  = Path("data/raw/test/test")

#Paths for processsed/reduced dataset
train_dir = Path("data/train")
test_dir = Path("data/test")


#move iamges into subfolders for cat and dog

for folder in [train_dir /"cats", train_dir / "dogs",
               test_dir / "cats", test_dir / "dogs",]:
    folder.mkdir(parents=True, exist_ok=True)


#Randomly takes 2,5000 cats and dog images from the data set to for the reduced training dataset

def copy_random_n_images(src_folder, dest_folder, class_name, n):

    class_files = list((src_folder/ class_name).glob("*.jpg")) #list all images
    selected_files = random.sample(class_files, n) #randomly  pick n
    for f in selected_files:
        shutil.copy(f, dest_folder /  class_name / f.name)

#Copy the random 2,5000 per class for training (cats and dogs)
copy_random_n_images(raw_train_dir, train_dir, "cats", n=2500)
copy_random_n_images(raw_train_dir, train_dir, "dogs", n=2500)

#Copy the random 1,000 per class for test (cats and dogs)
copy_random_n_images(raw_train_dir, train_dir, "cats", n=2500)
copy_random_n_images(raw_train_dir, train_dir, "dogs", n=2500)

#print result

print("REduced dataset created sucessfully")


SyntaxError: unterminated string literal (detected at line 9) (3465235522.py, line 9)